In [60]:
%cd C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor
from src.utils.cross_validation import CrossValidation
from src.models.random_forest_regression import RandomForestRegression

import pandas as pd
import xarray as xr
from pathlib import Path

C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor


In [72]:
ds = xr.load_dataset(r'C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor\data\intermediate\CV_results\performance_cube.nc')
ds = ds.squeeze()
da = ds.mse.sel(max_depth='50')**0.5
da

<xarray.DataArray 'mse' (cv_fold: 2, min_weight_fraction_leaf: 3)> Size: 48B
array([[0.04421336, 0.04519222, 0.0507363 ],
       [0.04409072, 0.04522831, 0.05049556]])
Coordinates:
  * cv_fold                   (cv_fold) int64 16B 0 1
    n_estimators              <U2 8B '30'
    max_depth                 <U2 8B '50'
    min_samples_split         <U1 4B '2'
    min_samples_leaf          <U1 4B '1'
    max_features              <U3 12B '1.0'
    max_samples               <U3 12B '1.0'
  * min_weight_fraction_leaf  (min_weight_fraction_leaf) <U6 72B '0.0' ... '0...

In [ ]:
ds = ds.squeeze()
da = ds.mse.sel(max_depth='50')

AttributeError: 'DataArray' object has no attribute 'mse'

In [66]:
ds

<xarray.DataArray 'mse' (cv_fold: 2, min_weight_fraction_leaf: 3)> Size: 48B
array([[0.00194968, 0.00204345, 0.00257417],
       [0.00193458, 0.00204491, 0.0025498 ]])
Coordinates:
  * cv_fold                   (cv_fold) int64 16B 0 1
    n_estimators              <U2 8B '30'
    max_depth                 <U2 8B '35'
    min_samples_split         <U1 4B '2'
    min_samples_leaf          <U1 4B '1'
    max_features              <U3 12B '1.0'
    max_samples               <U3 12B '1.0'
  * min_weight_fraction_leaf  (min_weight_fraction_leaf) <U6 72B '0.0' ... '0...